<a href="https://colab.research.google.com/github/benai9916/license-plate-detection-extraction-/blob/main/license_plate_detection_and_ORC.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Number plate detection

## Installation
Installing the Tensorflow Object Detection API became a lot easier with the relase of Tensorflow 2.

In [ ]:
!nvidia-smi

Sun Sep 27 07:44:12 2020       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 450.66       Driver Version: 418.67       CUDA Version: 10.1     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |                      |               MIG M. |
|===============================+======================+======================|
|   0  Tesla T4            Off  | 00000000:00:04.0 Off |                    0 |
| N/A   63C    P8    11W /  70W |      0MiB / 15079MiB |      0%      Default |
|                               |                      |                 ERR! |
+-------------------------------+----------------------+----------------------+
                                                                               
+-------

In [ ]:
# !pip install -U --pre tensorflow=="2.2.0"

!pip install tensorflow-gpu

# library to load file from google drive
!pip install gdown

In [ ]:
# Clone the tensorflow models repository if it doesn't already exist
import os
import pathlib


if "models" in pathlib.Path.cwd().parts:
  while "models" in pathlib.Path.cwd().parts:
    os.chdir('..')
    
elif not pathlib.Path('models').exists():
  !git clone --depth 1 https://github.com/tensorflow/models

Cloning into 'models'...
remote: Enumerating objects: 2198, done.
remote: Counting objects: 100% (2198/2198), done.
remote: Compressing objects: 100% (1900/1900), done.
remote: Total 2198 (delta 527), reused 959 (delta 272), pack-reused 0
Receiving objects: 100% (2198/2198), 30.43 MiB | 25.60 MiB/s, done.
Resolving deltas: 100% (527/527), done.


In [ ]:
# Install the Object Detection API

%%bash
cd models/research/
protoc object_detection/protos/*.proto --python_out=.
cp object_detection/packages/tf2/setup.py .
python -m pip install .

In [ ]:
#run model builder test

!python /content/models/research/object_detection/builders/model_builder_tf2_test.py

# Prepare data
After collecting the images we need to 
- label image, for this I am using [LabelImg](https://github.com/tzutalin/labelImg) - an free, open source graphical image annotation tool.
- create pbtxt file which will include no of class labels
- convert xml file into single csv
- create TFrecord file

In [ ]:
# download file to generate TFrecord

!wget https://gist.githubusercontent.com/benai9916/fe9b432640ce3153c5ed93b471d95896/raw/27dae42e83e48d56767d6888d8954686232135ba/generate_tfrecord.py

--2020-09-27 07:46:23--  https://gist.githubusercontent.com/benai9916/fe9b432640ce3153c5ed93b471d95896/raw/27dae42e83e48d56767d6888d8954686232135ba/generate_tfrecord.py
Resolving gist.githubusercontent.com (gist.githubusercontent.com)... 151.101.0.133, 151.101.64.133, 151.101.128.133, ...
Connecting to gist.githubusercontent.com (gist.githubusercontent.com)|151.101.0.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 3216 (3.1K) [text/plain]
Saving to: ‘generate_tfrecord.py’

generate_tfrecord.p 100%[===================>]   3.14K  --.-KB/s    in 0s      

2020-09-27 07:46:23 (71.3 MB/s) - ‘generate_tfrecord.py’ saved [3216/3216]



In [ ]:
# load file to convert xml to csv

!wget https://gist.githubusercontent.com/benai9916/8d6f4f3ce35e5063ae0487ce2a88d4db/raw/d666a4684102ca878da8c984480920885a17321d/xml_to_csv.py

--2020-09-27 07:46:23--  https://gist.githubusercontent.com/benai9916/8d6f4f3ce35e5063ae0487ce2a88d4db/raw/d666a4684102ca878da8c984480920885a17321d/xml_to_csv.py
Resolving gist.githubusercontent.com (gist.githubusercontent.com)... 151.101.0.133, 151.101.64.133, 151.101.128.133, ...
Connecting to gist.githubusercontent.com (gist.githubusercontent.com)|151.101.0.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 1198 (1.2K) [text/plain]
Saving to: ‘xml_to_csv.py’

xml_to_csv.py       100%[===================>]   1.17K  --.-KB/s    in 0s      

2020-09-27 07:46:23 (90.6 MB/s) - ‘xml_to_csv.py’ saved [1198/1198]



In [ ]:
# load dataset, file to generate TRrecord and xml_to_csv

if os.path.exists('images.zip'):
  print('File already exists')
else:
  !gdown --id 1kqqaf7ADtR0Dhn1OzE4yfWKH3UB4MVA7 --output images.zip

Downloading...
From: https://drive.google.com/uc?id=1kqqaf7ADtR0Dhn1OzE4yfWKH3UB4MVA7
To: /content/images.zip
21.8MB [00:00, 59.9MB/s]


In [ ]:
# unzip

!unzip images.zip

In [ ]:
pwd

'/content'

In [ ]:
# Convert image to RGB

from PIL import Image

for file in os.listdir('images/test'):
  if file.split('.')[1] == 'jpg':
    img = Image.open('images/test/'+ file)
    if img.mode != 'RGB':
      img.convert('RGB')


In [ ]:
for file in os.listdir('images/train'):
  if file.split('.')[1] == 'jpg':
    img = Image.open('images/train/'+ file)
    if img.mode != 'RGB':
      img.convert('RGB')

In [ ]:
# mak directory for csv

os.mkdir('labels')


# generate csv

!python xml_to_csv.py

In [ ]:
# make dir for tfrecord

os.mkdir('tfrecord')


# convert to csv file to TFrecord

!python generate_tfrecord.py --csv_input=labels/train_labels.csv --image_dir=images/train --output_path=tfrecord/train.record
!python generate_tfrecord.py --csv_input=labels/test_labels.csv --image_dir=images/test --output_path=tfrecord/test.record

2020-09-27 07:46:28.269043: I tensorflow/stream_executor/platform/default/dso_loader.cc:48] Successfully opened dynamic library libcudart.so.10.1
Successfully created the TFRecords: /content/tfrecord/train.record
2020-09-27 07:46:30.675173: I tensorflow/stream_executor/platform/default/dso_loader.cc:48] Successfully opened dynamic library libcudart.so.10.1
Successfully created the TFRecords: /content/tfrecord/test.record


In [ ]:
# specify paths

train_record_path = 'tfrecord/train.record'
test_record_path = 'tfrecord/test.record'
labelmap_path = 'label_map.pbtxt'

## Configuring training

There are many pre-trained object detection models available in the [model zoo](https://github.com/tensorflow/models/blob/master/research/object_detection/g3doc/tf2_detection_zoo.md). In order to train them using our custom data set, the models need to be restored in Tensorflow using their checkpoints (.ckpt files), which are records of previous model states.

I am going to use `ssd_mobilenet_v2_coco`

In [ ]:
# load pre trained

!wget http://download.tensorflow.org/models/object_detection/tf2/20200711/efficientdet_d0_coco17_tpu-32.tar.gz

!tar -xf efficientdet_d0_coco17_tpu-32.tar.gz



--2020-09-27 07:46:32--  http://download.tensorflow.org/models/object_detection/tf2/20200711/efficientdet_d0_coco17_tpu-32.tar.gz
Resolving download.tensorflow.org (download.tensorflow.org)... 74.125.195.128, 2607:f8b0:400e:c09::80
Connecting to download.tensorflow.org (download.tensorflow.org)|74.125.195.128|:80... connected.
HTTP request sent, awaiting response... 200 OK
Length: 30736482 (29M) [application/x-tar]
Saving to: ‘efficientdet_d0_coco17_tpu-32.tar.gz’

efficientdet_d0_coc 100%[===================>]  29.31M   137MB/s    in 0.2s    

2020-09-27 07:46:32 (137 MB/s) - ‘efficientdet_d0_coco17_tpu-32.tar.gz’ saved [30736482/30736482]



In [ ]:
fine_tune_checkpoint = 'efficientdet_d0_coco17_tpu-32/checkpoint/ckpt-0'


In [ ]:
# download the configuration file

!wget https://raw.githubusercontent.com/tensorflow/models/master/research/object_detection/configs/tf2/ssd_efficientdet_d0_512x512_coco17_tpu-8.config

base_config_path = 'ssd_efficientdet_d0_512x512_coco17_tpu-8.config'


--2020-09-27 07:46:33--  https://raw.githubusercontent.com/tensorflow/models/master/research/object_detection/configs/tf2/ssd_efficientdet_d0_512x512_coco17_tpu-8.config
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 151.101.0.133, 151.101.64.133, 151.101.128.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|151.101.0.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 4630 (4.5K) [text/plain]
Saving to: ‘ssd_efficientdet_d0_512x512_coco17_tpu-8.config’

ssd_efficientdet_d0 100%[===================>]   4.52K  --.-KB/s    in 0s      

2020-09-27 07:46:33 (58.3 MB/s) - ‘ssd_efficientdet_d0_512x512_coco17_tpu-8.config’ saved [4630/4630]



In [ ]:
batch_size = 16
num_steps = 8000
num_eval_steps = 1000

In `config` file we need to change:
- Since we’re only trying to license plate, change num_classes to 1
- `fine_tune_checkpoint` tells the model which checkpoint file to use. Set this to `checkpoints/model.ckpt`
- The model also needs to know where the TFRecord files and label maps are for both training and validation sets. Since our `train.record` and test.record are saved in tf_record folder, our config should reflect that:
- change the `batch_size`, `num_step`, `num_eval_steps`

In [ ]:
import re

with open(base_config_path) as f:
    config = f.read()

with open('model.config', 'w') as f:
  
  # Set labelmap path
  config = re.sub('label_map_path: ".*?"', 
             'label_map_path: "{}"'.format(labelmap_path), config)
  
  # Set fine_tune_checkpoint path
  config = re.sub('fine_tune_checkpoint: ".*?"',
                  'fine_tune_checkpoint: "{}"'.format(fine_tune_checkpoint), config)
  
  # Set train tf-record file path
  config = re.sub('(input_path: ".*?)(PATH_TO_BE_CONFIGURED/train)(.*?")', 
                  'input_path: "{}"'.format(train_record_path), config)
  
  # Set test tf-record file path
  config = re.sub('(input_path: ".*?)(PATH_TO_BE_CONFIGURED/val)(.*?")', 
                  'input_path: "{}"'.format(test_record_path), config)
  
  # Set number of classes.
  config = re.sub('num_classes: [0-9]+',
                  'num_classes: {}'.format(1), config)
  
  # Set batch size
  config = re.sub('batch_size: [0-9]+',
                  'batch_size: {}'.format(batch_size), config)
  
  # Set training steps
  config = re.sub('num_steps: [0-9]+',
                  'num_steps: {}'.format(num_steps), config)
  
  # Set fine-tune checkpoint type to detection
  config = re.sub('fine_tune_checkpoint_type: "classification"', 
             'fine_tune_checkpoint_type: "{}"'.format('detection'), config)
  
  f.write(config)

In [ ]:
%cat model.config

In [ ]:
model_dir = 'training/'
pipeline_config_path = 'model.config'

## Train detector

In [ ]:
!python /content/models/research/object_detection/model_main_tf2.py \
    --pipeline_config_path={pipeline_config_path} \
    --model_dir={model_dir} \
    --alsologtostderr \
    --num_train_steps={num_steps} \
    --sample_1_of_n_eval_examples=1 \
    --num_eval_steps={num_eval_steps}

In [ ]:
%load_ext tensorboard
%tensorboard --logdir '/content/training/train'

The below code cell adds a line to the tf_utils.py file. This is a temporary fix to a [exporting issue](https://github.com/tensorflow/models/issues/8841) occuring when using the OD API with Tensorflow 2. This code will be removed as soon as the OD Team puts out a fix.

All credit goes to Github user [Jacobsolawetz](https://github.com/Jacobsolawetz), who provided this [temporary fix](https://github.com/tensorflow/models/issues/8841#issuecomment-657647648).

In [ ]:
with open('/usr/local/lib/python3.6/dist-packages/tensorflow/python/keras/utils/tf_utils.py') as f:
    tf_utils = f.read()

with open('/usr/local/lib/python3.6/dist-packages/tensorflow/python/keras/utils/tf_utils.py', 'w') as f:
  # Set labelmap path
  throw_statement = "raise TypeError('Expected Operation, Variable, or Tensor, got ' + str(x))"
  tf_utils = tf_utils.replace(throw_statement, "if not isinstance(x, str):" + throw_statement)
  f.write(tf_utils)

In [ ]:
# generate an inference graph

output_directory = 'inference_graph'

!python /content/models/research/object_detection/exporter_main_v2.py \
    --trained_checkpoint_dir {model_dir} \
    --output_directory {output_directory} \
    --pipeline_config_path {pipeline_config_path}

In [ ]:
# download the graph

from google.colab import files
files.download(f'/content/{output_directory}/saved_model/saved_model.pb') 

# Test model on test images

based on [Object Detection API Demo](https://github.com/tensorflow/models/blob/master/research/object_detection/colab_tutorials/object_detection_tutorial.ipynb) and [Inference from saved model tf2 colab](https://github.com/tensorflow/models/blob/master/research/object_detection/colab_tutorials/inference_from_saved_model_tf2_colab.ipynb).

In [ ]:
# Import libraries

import io
import os
import scipy.misc
import numpy as np
import six
import time
import glob
from IPython.display import display

from six import BytesIO

import matplotlib
import matplotlib.pyplot as plt
from PIL import Image, ImageDraw, ImageFont

import tensorflow as tf
from object_detection.utils import ops as utils_ops
from object_detection.utils import label_map_util
from object_detection.utils import visualization_utils as vis_util

%matplotlib inline

In [ ]:
def load_image_into_numpy_array(path):
  """Load an image from file into a numpy array.

  Puts image into numpy array to feed into tensorflow graph.
  Note that by convention we put it into a numpy array with shape
  (height, width, channels), where channels=3 for RGB.

  Args:
    path: a file path (this can be local or on colossus)

  Returns:
    uint8 numpy array with shape (img_height, img_width, 3)
  """
  img_data = tf.io.gfile.GFile(path, 'rb').read()
  image = Image.open(BytesIO(img_data))
  (im_width, im_height) = image.size

  return np.array(image.getdata()).reshape((im_height, im_width, 3)).astype(np.uint8)

In [ ]:
# indexing the category
# labelmap_path = 'label_map.pbtxt'

category_index = label_map_util.create_category_index_from_labelmap(labelmap_path, use_display_name=True)

In [ ]:
# load model

tf.keras.backend.clear_session()
model = tf.saved_model.load(f'/content/{output_directory}/saved_model')

In [ ]:
def run_inference_for_single_image(model, image):
  image = np.asarray(image)
  
  # The input needs to be a tensor, convert it using `tf.convert_to_tensor`.
  input_tensor = tf.convert_to_tensor(image)

  # The model expects a batch of images, so add an axis with `tf.newaxis`.
  input_tensor = input_tensor[tf.newaxis,...]

  # Run inference
  model_fn = model.signatures['serving_default']
  output_dict = model_fn(input_tensor)

  # All outputs are batches tensors.
  # Convert to numpy arrays, and take index [0] to remove the batch dimension.
  # We're only interested in the first num_detections.
  num_detections = int(output_dict.pop('num_detections'))
  output_dict = {key:value[0, :num_detections].numpy() 
                 for key,value in output_dict.items()}
  output_dict['num_detections'] = num_detections

  # detection_classes should be ints.
  output_dict['detection_classes'] = output_dict['detection_classes'].astype(np.int64)
   
  # Handle models with masks:
  if 'detection_masks' in output_dict:
    # Reframe the the bbox mask to the image size.
    detection_masks_reframed = utils_ops.reframe_box_masks_to_image_masks(
              output_dict['detection_masks'], output_dict['detection_boxes'],
               image.shape[0], image.shape[1])      
    detection_masks_reframed = tf.cast(detection_masks_reframed > 0.5,
                                       tf.uint8)
    output_dict['detection_masks_reframed'] = detection_masks_reframed.numpy()

    
  return output_dict

In [ ]:
# for image_path in glob.glob('microcontroller-detection/test/*.jpg'):

image_np = load_image_into_numpy_array('images/test/3.jpg')
output_dict = run_inference_for_single_image(model, image_np)
vis_util.visualize_boxes_and_labels_on_image_array(
    image_np,
    output_dict['detection_boxes'],
    output_dict['detection_classes'],
    output_dict['detection_scores'],
    category_index,
    instance_masks=output_dict.get('detection_masks_reframed', None),
    use_normalized_coordinates=True,
    line_thickness=4)

display(Image.fromarray(image_np))